# Create files in different prediction scenarii
* Random
* Unseen_drugs
* Unseen_targets
* Orphan

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load LCIdb

The dataset can be downloaded in Zenodo \url{https://zenodo.org/records/10731713} as LCIdb_v1.1.csv.

In [7]:
base_name_p = 'LCIdb_v1.1.csv'
 
df_p = pd.read_csv(base_name_p,low_memory=False)
df_p.head()


,smiles,fasta,ChEMBL ID,PubChem ID,IUPHAR ID,Ligand names,Target,uniprot,mean,mean pIC50,...,min pIC50,min pKi,min pKd,max,max pIC50,max pKi,max pKd,score,indsmiles,indfasta
0,BrC1CCN(Cc2ccc(OCCCN3CCCCC3)cc2)CC1,MERAPPDGPLNASGALAGEAAAAGGARGFSAAWTAVLAALMALLIV...,CHEMBL472466,44592131.0,NaN,4-bromo-1-(4-(3-(piperidin-1-yl)propoxy)benzyl...,hrh3,Q9Y5N1,9.7,NaN,...,NaN,9.7,NaN,9.7,NaN,9.7,NaN,1.0,0.0,0.0
1,BrCC(Br)C1CCC(Br)C(Br)C1,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,CHEMBL375107,18728.0,NaN,"1,2-dibromo-4-(1,2-dibromoethyl)cyclohexane\nn...",ar,P10275,7.4,7.4,...,7.4,NaN,NaN,7.4,7.4,NaN,NaN,1.0,1.0,1.0
2,BrC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,MAPPSEETPLIPQRSCSLLSTEAGALHVLLPARGPGPPQRLSFSFG...,CHEMBL3918308,136641870.0,NaN,"us9216999, 74",jak3,P52333,7.1,7.1,...,7.1,NaN,NaN,7.1,7.1,NaN,NaN,1.0,2.0,2.0
3,BrC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...,CHEMBL3918308,136641870.0,NaN,"us9216999, 74",jak2,O60674,8.5,8.5,...,8.5,NaN,NaN,8.5,8.5,NaN,NaN,1.0,2.0,3.0
4,BrC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,MPLRHWGMARGSKPVGDGAQPMAAMGGLKVLLHWAGPGGGEPWVTF...,CHEMBL3918308,136641870.0,NaN,"us9216999, 74",tyk2,P29597,8.6,8.6,...,8.6,NaN,NaN,8.6,8.6,NaN,NaN,1.0,2.0,4.0


#### Drugs

In [9]:
#list of smiles strings
dict_ind2smiles = df_p[['indsmiles','smiles']].set_index('indsmiles').to_dict()['smiles']
#Sort the indexes according to the ind2mol
dict_ind2smiles = {k:dict_ind2smiles[k] for k in sorted(dict_ind2smiles.keys())}
print(len(dict_ind2smiles))
dict_smiles2ind = {v:k for k,v in dict_ind2smiles.items()}
smiles = list(dict_ind2smiles.values())
print("# drugs",len(smiles))

274515
# drugs 274515


#### Proteins

In [10]:
#list of fasta strings
dict_ind2fasta = df_p[['indfasta','fasta']].set_index('indfasta').to_dict()['fasta']
# Sort the indexes according to the ind2fasta
dict_ind2fasta = {k:dict_ind2fasta[k] for k in sorted(dict_ind2fasta.keys())}
print(len(dict_ind2fasta))
dict_fasta2ind = {v:k for k,v in dict_ind2fasta.items()}
fastas = list(dict_ind2fasta.values())
print("# targets ",len(fastas))

2069
# targets  2069


#### number of interactions

In [11]:
df_p["score"].value_counts()

1.0    402538
0.5    265006
0.0      8296
Name: score, dtype: int64

## list of target/drug indices with only interaction 1

In [12]:
# targets indices
J = df_p['indfasta'].values
print(len(J))
# drugs indices
I = df_p['indsmiles'].values
print(len(I))
J

676125
676125


array([  0.,   1.,   2., ..., 180.,  87.,  50.])

### Make train/val/test Random (S1)

In [14]:
from komet import make_train_test_val_S1

train, test, val = make_train_test_val_S1(df_p,0.8,0.12)
test.shape[0]/train.shape[0], val.shape[0]/train.shape[0]

train 322030 test 48304 val 32204
train (644060, 3)
nb of interactions + in test 48304
number of real interactions - in test 6664
number of interactions - in test 48304
test (96608, 3)
nb of interactions + in val 32204
number of real interactions - in val 0
number of interactions - in val 32204
val (64408, 3)
Train/test/val datasets prepared.


(0.1499984473496258, 0.10000310530074838)

In [15]:
print("number of proteins in train", len(set(train[:,0])))
print("number of proteins in test", len(set(test[:,0])))
print("number of proteins in val", len(set(val[:,0])))
print("*"*50)
print("number of drugs in train", len(set(train[:,1])))
print("number of drugs in test", len(set(test[:,1])))
print("number of drugs in val", len(set(val[:,1])))
print("*"*50)
print("number of interactions in train", train.shape[0])
print("number of interactions in test", test.shape[0])
print("number of interactions in val", val.shape[0])
print("*"*50)
print("number of interactions + in train", train[:,2].sum())
print("number of interactions + in test", test[:,2].sum())
print("number of interactions + in val", val[:,2].sum())
print("*"*50)
print("number of interactions - in train", train.shape[0]-train[:,2].sum())
print("number of interactions - in test", test.shape[0]-test[:,2].sum())
print("number of interactions - in val", val.shape[0]-val[:,2].sum())

number of proteins in train 2015
number of proteins in test 2069
number of proteins in val 2069
**************************************************
number of drugs in train 231276
number of drugs in test 79170
number of drugs in val 57272
**************************************************
number of interactions in train 644060
number of interactions in test 96608
number of interactions in val 64408
**************************************************
number of interactions + in train 322030
number of interactions + in test 48304
number of interactions + in val 32204
**************************************************
number of interactions - in train 322030
number of interactions - in test 48304
number of interactions - in val 32204


#### Make a dataframe with smiles/fasta to save in .csv

In [16]:
# dataframes
df_train = pd.DataFrame(train,columns=['indfasta','indsmiles','label'])
print("train : #target/drugs : ",df_train["indfasta"].nunique(),df_train["indsmiles"].nunique())
df_test = pd.DataFrame(test,columns=['indfasta','indsmiles','label'])
print("test : #target/drugs : ",df_test["indfasta"].nunique(),df_test["indsmiles"].nunique())
df_val = pd.DataFrame(val,columns=['indfasta','indsmiles','label'])
print("val : #target/drugs : ",df_val["indfasta"].nunique(),df_val["indsmiles"].nunique())

# add smiles and fasta to the dataframes
df_train_S1 = df_train.merge(df_p[["indfasta","fasta"]].drop_duplicates(),on="indfasta")
df_train_S1 = df_train_S1[["indsmiles","fasta","label"]]
df_train_S1 = df_train_S1.merge(df_p[["indsmiles","smiles"]].drop_duplicates(),on="indsmiles")
df_train_S1 = df_train_S1[["smiles","fasta","label"]]
df_train_S1.columns = ["SMILES","Target Sequence","Label"]

df_test_S1 = df_test.merge(df_p[["indfasta","fasta"]].drop_duplicates(),on="indfasta")
df_test_S1 = df_test_S1[["indsmiles","fasta","label"]]
df_test_S1 = df_test_S1.merge(df_p[["indsmiles","smiles"]].drop_duplicates(),on="indsmiles")
df_test_S1 = df_test_S1[["smiles","fasta","label"]]
df_test_S1.columns = ["SMILES","Target Sequence","Label"]

df_val_S1 = df_val.merge(df_p[["indfasta","fasta"]].drop_duplicates(),on="indfasta")
df_val_S1 = df_val_S1[["indsmiles","fasta","label"]]
df_val_S1 = df_val_S1.merge(df_p[["indsmiles","smiles"]].drop_duplicates(),on="indsmiles")
df_val_S1 = df_val_S1[["smiles","fasta","label"]]
df_val_S1.columns = ["SMILES","Target Sequence","Label"]

train : #prot/durgs :  2015 231276
test : #prot/durgs :  2069 79170
val : #prot/durgs :  2069 57272


In [17]:
print("train")
df_train['label'].value_counts() 

train


1    322030
0    322030
Name: label, dtype: int64

In [18]:
print("test")
df_test['label'].value_counts()

test


1    48304
0    48304
Name: label, dtype: int64

In [19]:
print("val")
df_val['label'].value_counts()

val


1    32204
0    32204
Name: label, dtype: int64

In [ ]:
# for each pair, we look if the protein is in the train and the test
for elt in test:
    for x in train:
        if elt[0]==x[0]:
            print(elt,x)

In [21]:
# for each pair, we look if the drug is in the train and the test
for elt in test:
    for x in train:
        if elt[1]==x[1]:
            print(elt,x)

[ 1264 40293     1] [  186 40293     0]
[ 1264 40293     1] [  837 40293     1]
[   207 270858      1] [   208 270858      1]
[   207 270858      1] [    86 270858      0]
[    25 103530      1] [    44 103530      1]
[    25 103530      1] [   115 103530      0]
[    25 239743      1] [    44 239743      1]
[    25 239743      1] [   162 239743      0]
[  120 10888     1] [   51 10888     1]
[  120 10888     1] [   25 10888     0]
[  120 10888     1] [  368 10888     0]
[  120 10888     1] [  287 10888     1]
[  194 81092     1] [  324 81092     0]
[  194 81092     1] [  193 81092     1]
[   28 95164     1] [  226 95164     0]
[   28 95164     1] [  452 95164     1]
[  566 16455     1] [  286 16455     0]
[  566 16455     1] [  207 16455     0]
[  566 16455     1] [  172 16455     0]
[  566 16455     1] [  391 16455     0]
[  566 16455     1] [   49 16455     1]
[  566 16455     1] [  275 16455     1]
[  566 16455     1] [  564 16455     1]
[  566 16455     1] [  590 16455     1]
[  1

KeyboardInterrupt: 